## Step 3. Data Preprocessing & Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

# Load raw gzipped CSV (this avoids datetime conversion issues)
file_path = "../data/raw/retailrocket_events.csv.gz"
df = pd.read_csv(file_path, compression='gzip', low_memory=False)
print(f"Loaded raw dataframe shape: {df.shape}")
print(df.head())

### 1. Duplicate data for preprocessing safety
df_preproc = df.copy()
print(f"Step 1: Data duplicated for preprocessing. Number of rows: {len(df_preproc)}")

### 2. Handle missing values in item metadata (categories, descriptions)
item_props_1 = pd.read_csv("../data/raw/item_properties_part1.csv")
item_props_2 = pd.read_csv("../data/raw/item_properties_part2.csv")

item_props = pd.concat([item_props_1, item_props_2])
duplicates = item_props.duplicated(subset=['itemid', 'property'], keep=False)
print(f"Duplicate (itemid, property) pairs: {duplicates.sum()}")
print("Sample duplicate rows:")
print(item_props[duplicates].head(10))

# Aggregate duplicates by first occurrence
item_props_agg = item_props.groupby(['itemid', 'property'])['value'].first().reset_index()

# Pivot to wide format
item_props_wide = item_props_agg.pivot(index='itemid', columns='property', values='value').reset_index()

# Fill missing values
if 'category' in item_props_wide.columns:
    item_props_wide['category'] = item_props_wide['category'].fillna('unknown')
else:
    item_props_wide['category'] = 'unknown'

if 'description' in item_props_wide.columns:
    item_props_wide['description'] = item_props_wide['description'].fillna('')
else:
    item_props_wide['description'] = ''

print("\nMissing values after filling:")
print(item_props_wide.isna().sum())

item_props_wide.to_parquet("../data/processed/item_properties_wide.parquet")
print(f"Cleaned item properties saved. Shape: {item_props_wide.shape}")

### 3. Extract timestamp features from df_preproc
print("\nExtracting datetime features from 'timestamp'...")
df_preproc['datetime'] = pd.to_datetime(df_preproc['timestamp'], unit='ms', errors='coerce')
print(df_preproc[['timestamp', 'datetime']].head())

invalid_datetime_count = df_preproc['datetime'].isna().sum()
print(f"Rows with invalid datetime: {invalid_datetime_count}")
df_preproc = df_preproc.dropna(subset=['datetime'])
print(f"Rows after dropping invalid datetime: {len(df_preproc)}")

# Extract datetime components
df_preproc['day'] = df_preproc['datetime'].dt.day
df_preproc['hour'] = df_preproc['datetime'].dt.hour
df_preproc['weekday'] = df_preproc['datetime'].dt.weekday
df_preproc['month'] = df_preproc['datetime'].dt.month

print("Datetime features extracted:")
print(df_preproc[['visitorid', 'datetime', 'day', 'hour', 'weekday', 'month']].head())

# Calculate recency (days since last event per user)
last_event = df_preproc.groupby('visitorid')['datetime'].max().reset_index()
last_event.rename(columns={'datetime': 'last_event_time'}, inplace=True)
print("Last event per visitor sample:")
print(last_event.head())

df_preproc = df_preproc.merge(last_event, on='visitorid', how='left')
df_preproc['recency_days'] = (df_preproc['last_event_time'] - df_preproc['datetime']).dt.days
print("Sample with recency_days:")
print(df_preproc[['visitorid', 'datetime', 'last_event_time', 'recency_days']].head())

# Save preprocessed dataframe for further steps
df_preproc.to_parquet("../data/processed/df_preprocessed.parquet")
print("Step 3 preprocessing completed and saved.")

### 4. Item metadata categorical encoding
print("\nEncoding categorical item properties...")

item_props = pd.read_csv("../data/raw/item_properties_part1.csv", usecols=['itemid', 'property', 'value'])
print(f"Loaded item properties rows: {len(item_props)}")

dupes_count = item_props.duplicated(subset=['itemid', 'property']).sum()
print(f"Duplicate (itemid, property) pairs: {dupes_count}")

# Aggregate duplicates by mode of 'value'
item_props_agg = (
    item_props.groupby(['itemid', 'property'])['value']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])
    .reset_index()
)

print(f"Rows after aggregation: {len(item_props_agg)}")

# Pivot to wide format again for encoding
item_props_wide = item_props_agg.pivot(index='itemid', columns='property', values='value').reset_index()
print("Pivoted to wide format.")

# Select categorical columns (example: 'categoryid' and 'available')
categorical_cols = [col for col in ['categoryid', 'available'] if col in item_props_wide.columns]
print(f"Categorical columns for encoding: {categorical_cols}")

# One-hot encode if any categorical columns found
if categorical_cols:
    item_props_wide[categorical_cols] = item_props_wide[categorical_cols].fillna('missing').astype(str)
    item_props_encoded = pd.get_dummies(item_props_wide, columns=categorical_cols, dummy_na=False)
else:
    item_props_encoded = item_props_wide.copy()
    print("No categorical columns found for encoding.")

print(f"Encoded item properties shape: {item_props_encoded.shape}")
item_props_encoded.to_pickle("../data/processed/item_properties_encoded.pkl")
print("Encoded item properties saved.")

### 5. Remove outliers (users/items with fewer than 5 interactions)
print("\nRemoving users and items with fewer than 5 interactions...")

user_interactions = df_preproc['visitorid'].value_counts()
print(f"Number of users before filtering: {len(user_interactions)}")
valid_users = user_interactions[user_interactions >= 5].index
df_filtered = df_preproc[df_preproc['visitorid'].isin(valid_users)]

item_interactions = df_filtered['itemid'].value_counts()
print(f"Number of items before filtering: {len(item_interactions)}")
valid_items = item_interactions[item_interactions >= 5].index
df_filtered = df_filtered[df_filtered['itemid'].isin(valid_items)]

print(f"Data shape after filtering outliers: {df_filtered.shape}")

# Save filtered dataset for modeling
cleaned_path = '../data/processed/df_filtered.pkl'
df_filtered.to_pickle(cleaned_path)
print(f"Filtered data saved to {cleaned_path}")

print("Step 3 Data Preprocessing & Feature Engineering completed.")


Loaded raw dataframe shape: (2756101, 5)
       timestamp  visitorid event  itemid  transactionid
0  1433221332117     257597  view  355908            NaN
1  1433224214164     992329  view  248676            NaN
2  1433221999827     111016  view  318965            NaN
3  1433221955914     483717  view  253185            NaN
4  1433221337106     951259  view  367447            NaN
Step 1: Data duplicated for preprocessing. Number of rows: 2756101
Duplicate (itemid, property) pairs: 8786767
Sample duplicate rows:
        timestamp  itemid   property                            value
1   1441508400000  206783        888          1116713 960601 n277.200
2   1439089200000  395014        400  n552.000 639502 n720.000 424566
3   1431226800000   59481        790                       n15360.000
5   1436065200000  285026  available                                0
9   1434250800000   98113        451                  1141052 n48.000
10  1439089200000  450113        888           1038400 45956 n5